<a href="https://colab.research.google.com/github/Ayushsrinivas7/LLM_LEARNINGS/blob/main/Quantized_ANN_FINAL_VERSION_FOR_SUMANTHA_SIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy
!pip install nnfs

In [ ]:
!pip install tensorflow==2.8.0
!pip install larq==0.12.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 34.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: goo

In [ ]:
 from tensorflow.python.ops.numpy_ops import np_config
 np_config.enable_numpy_behavior()
 import larq as lq

In [ ]:
def quantize_weights( matrix  , k_bit ):
  quantizer = lq.quantizers.DoReFaQuantizer(k_bit= k_bit, mode="weights")
  return quantizer(matrix)

def quantize_activation( matrix , k_bit):
  quantizer = lq.quantizers.DoReFaQuantizer(k_bit=k_bit , mode="activations")
  return quantizer(matrix )

In [ ]:
import nnfs
from nnfs.datasets import spiral_data
import matplotlib.pyplot as plt
import numpy as np
import random

In [ ]:
# Common loss class
class Loss:
 # Calculates the data and regularization losses
 # given model output and ground truth values
 def calculate(self, output, y):
  # Calculate sample losses
  sample_losses = self.forward(output, y)
  # Calculate mean loss
  data_loss = np.mean(sample_losses)
  # Return loss
  return data_loss

In [ ]:
class Loss_CategoricalCrossentropy(Loss):
    # Forward pass
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = len(y_pred)

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values -
        # only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
                range(samples), y_true ]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(
               y_pred_clipped * y_true,
                axis=1
            )

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # We'll use the first sample to count them
        labels = len(dvalues[0])

        # If labels are sparse, turn them into one-hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate gradient
        self.dinputs = -y_true / dvalues
        # Normalize gradient
        self.dinputs = self.dinputs / samples

In [ ]:
# Softmax activation
class Activation_Softmax:
 # Forward pass
 def forward(self, inputs):
 # Get unnormalized probabilities
  exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
 # Normalize them for each sample
  probabilities = exp_values / np.sum(exp_values, axis=1,keepdims=True)
  self.output = probabilities

In [ ]:
# Softmax classifier - combined Softmax activation
# and cross-entropy loss for faster backward step
class Activation_Softmax_Loss_CategoricalCrossentropy:
    # Creates activation and loss function objects
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossentropy()

    # Forward pass
    def forward(self, inputs, y_true):
        # Output layer's activation function
        self.activation.forward(inputs)
        # Set the output
        self.output = self.activation.output
        # Calculate and return loss value
        return self.loss.calculate(self.output, y_true)

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # If labels are one-hot encoded,
        # turn them into discrete values
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        # Copy so we can safely modify
        self.dinputs = dvalues.copy()
        # Calculate gradient
        self.dinputs[range(samples), y_true] -= 1
        # Normalize gradient
        self.dinputs = self.dinputs / samples

In [ ]:
import numpy as np

# Adam optimizer
class Optimizer_Adam:
    # Initialize optimizer - set settings
    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7, beta_1=0.9, beta_2=0.999):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2

    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    # Update parameters
    def update_params(self, layer):
        # If layer does not contain cache arrays, create them filled with zeros
        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)
            layer.bias_cache = np.zeros_like(layer.biases)

        # Update momentum with current gradients
        layer.weight_momentums = self.beta_1 * layer.weight_momentums + (1 - self.beta_1) * layer.dweights
        layer.bias_momentums = self.beta_1 * layer.bias_momentums + (1 - self.beta_1) * layer.dbiases

        # Get corrected momentum
        # self.iteration is 0 at first pass and we need to start with 1 here
        weight_momentums_corrected = layer.weight_momentums / (1 - self.beta_1 ** (self.iterations + 1))
        bias_momentums_corrected = layer.bias_momentums / (1 - self.beta_1 ** (self.iterations + 1))

        # Update cache with squared current gradients
        layer.weight_cache = self.beta_2 * layer.weight_cache + (1 - self.beta_2) * layer.dweights**2
        layer.bias_cache = self.beta_2 * layer.bias_cache + (1 - self.beta_2) * layer.dbiases**2

        # Get corrected cache
        weight_cache_corrected = layer.weight_cache / (1 - self.beta_2 ** (self.iterations + 1))
        bias_cache_corrected = layer.bias_cache / (1 - self.beta_2 ** (self.iterations + 1))

        # Vanilla SGD parameter update + normalization with square rooted cache
        layer.weights += -self.current_learning_rate * weight_momentums_corrected / (np.sqrt(weight_cache_corrected) + self.epsilon)
        layer.biases += -self.current_learning_rate * bias_momentums_corrected / (np.sqrt(bias_cache_corrected) + self.epsilon)

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1

In [ ]:
# Dense layer
class Quant_Layer_Dense:
    # Layer initialization
    def __init__(self, n_inputs, n_neurons):
        # Initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        self.qw = self.weights.copy()
        self.qb = self.biases.copy()
    # Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
        # Calculate output values from input ones, weights and biases
        self.output = np.dot(inputs, self.qw) + self.qb

    # Backward pass
    def backward(self, dvalues):
        # Gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.qw.T)

In [ ]:
# ReLU activation
class Quant_Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
        # Calculate output values from inputs
        self.output = np.maximum(0, inputs)
        # ---> # self.output = quantize_activation(self.output , 8 )

    # Backward pass
    def backward(self, dvalues):
        # Since we need to modify the original variable,
        # let’s make a copy of values first
        self.dinputs = dvalues.copy()
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

normal quant

In [ ]:
import numpy as np

# Adam optimizer
class Quant_Optimizer_Adam:
    # Initialize optimizer - set settings
    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7, beta_1=0.9, beta_2=0.999):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2

    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    # Update parameters
    def update_params(self, layer):
        # If layer does not contain cache arrays, create them filled with zeros
        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)
            layer.bias_cache = np.zeros_like(layer.biases)

        # Update momentum with current gradients
        layer.weight_momentums = self.beta_1 * layer.weight_momentums + (1 - self.beta_1) * layer.dweights
        layer.bias_momentums = self.beta_1 * layer.bias_momentums + (1 - self.beta_1) * layer.dbiases

        # Get corrected momentum
        # self.iteration is 0 at first pass and we need to start with 1 here
        weight_momentums_corrected = layer.weight_momentums / (1 - self.beta_1 ** (self.iterations + 1))
        bias_momentums_corrected = layer.bias_momentums / (1 - self.beta_1 ** (self.iterations + 1))

        # Update cache with squared current gradients
        layer.weight_cache = self.beta_2 * layer.weight_cache + (1 - self.beta_2) * layer.dweights**2
        layer.bias_cache = self.beta_2 * layer.bias_cache + (1 - self.beta_2) * layer.dbiases**2

        # Get corrected cache
        weight_cache_corrected = layer.weight_cache / (1 - self.beta_2 ** (self.iterations + 1))
        bias_cache_corrected = layer.bias_cache / (1 - self.beta_2 ** (self.iterations + 1))

        # Vanilla SGD parameter update + normalization with square rooted cache
        layer.qw += -self.current_learning_rate * weight_momentums_corrected / (np.sqrt(weight_cache_corrected) + self.epsilon)
        layer.qb += -self.current_learning_rate * bias_momentums_corrected / (np.sqrt(bias_cache_corrected) + self.epsilon)
        layer.weights = layer.qw
        layer.biases = layer.qb
        # layer.qw = quantize_weights( layer.weights , 8)
        # layer.qb = np.clip(layer.biases , 0 , 1)

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1


wei , bias update quant

In [ ]:
import numpy as np

# Adam optimizer
class Quant2_Optimizer_Adam:
    # Initialize optimizer - set settings
    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7, beta_1=0.9, beta_2=0.999 , k_bit = 64 ):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.k_bit = k_bit

    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * (1. / (1. + self.decay * self.iterations))

    # Update parameters
    def update_params(self, layer):
        # If layer does not contain cache arrays, create them filled with zeros
        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)
            layer.bias_cache = np.zeros_like(layer.biases)

        # Update momentum with current gradients
        layer.weight_momentums = self.beta_1 * layer.weight_momentums + (1 - self.beta_1) * layer.dweights
        layer.bias_momentums = self.beta_1 * layer.bias_momentums + (1 - self.beta_1) * layer.dbiases

        # Get corrected momentum
        # self.iteration is 0 at first pass and we need to start with 1 here
        weight_momentums_corrected = layer.weight_momentums / (1 - self.beta_1 ** (self.iterations + 1))
        bias_momentums_corrected = layer.bias_momentums / (1 - self.beta_1 ** (self.iterations + 1))

        # Update cache with squared current gradients
        layer.weight_cache = self.beta_2 * layer.weight_cache + (1 - self.beta_2) * layer.dweights**2
        layer.bias_cache = self.beta_2 * layer.bias_cache + (1 - self.beta_2) * layer.dbiases**2

        # Get corrected cache
        weight_cache_corrected = layer.weight_cache / (1 - self.beta_2 ** (self.iterations + 1))
        bias_cache_corrected = layer.bias_cache / (1 - self.beta_2 ** (self.iterations + 1))

        # Vanilla SGD parameter update + normalization with square rooted cache
        layer.qw += -self.current_learning_rate * weight_momentums_corrected / (np.sqrt(weight_cache_corrected) + self.epsilon)
        layer.qb += -self.current_learning_rate * bias_momentums_corrected / (np.sqrt(bias_cache_corrected) + self.epsilon)
        layer.weights = layer.qw
        layer.biases = layer.qb
        layer.qw = quantize_weights( layer.weights ,self.k_bit  )
        layer.qb = quantize_activation(layer.biases , self.k_bit )

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1


activation quantize high

In [ ]:
# ReLU activation
class Quant2_Activation_ReLU:
    # Forward pass
    def forward(self, inputs , k_bit = 64  ):
        # Remember input values
        self.inputs = inputs
        self.k_bit = k_bit
        # Calculate output values from inputs
        self.output = np.maximum(0, inputs)
        self.output = quantize_activation(self.output , self.k_bit  )

    # Backward pass
    def backward(self, dvalues):
        # Since we need to modify the original variable,
        # let’s make a copy of values first
        self.dinputs = dvalues.copy()
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

In [ ]:
from keras.datasets import mnist


NORMAL TRAINING

In [ ]:
# Preprocess the MNIST data to resize from 28x28 to 12x12
from tensorflow.keras.utils import to_categorical

def preprocess_mnist(images):
    # Crop the images from 28x28 to 24x24 by removing 2 rows/columns from edges
    cropped_images = images[:, 2:26, 2:26]
    # Downsample by taking alternate pixels (24x24 to 12x12)
    downsampled_images = cropped_images[:, ::2, ::2]
    # Flatten the images into vectors of size 12x12 = 144
    flattened_images = downsampled_images.reshape(images.shape[0], -1)
    return flattened_images


(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = preprocess_mnist(X_train)
X_test = preprocess_mnist(X_test)

# Normalize pixel values to range [0, 1]
X_train = X_train / 255.0
X_test = X_test / 255.0

# One-hot encode labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# checking the shapes
print(X_train.shape)
print(y_train.shape)
print(y_train[0])
# Neural network setup
dense1 = Quant_Layer_Dense(144, 64)  # Input size is 12x12 = 144
activation1 = Quant_Activation_ReLU()
dense2 = Quant_Layer_Dense(64, 64)
activation2 = Quant_Activation_ReLU()
dense3 =  Quant_Layer_Dense(64, 10)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = Quant_Optimizer_Adam(learning_rate=0.001, decay=1e-6)

# Training loop


for epoch in range(1501):
    # Perform a forward pass of our training data through this layer
    dense1.forward(X_train)

    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)

    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
    dense3.forward(activation2.output)


    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense3.output, y_train)

    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y_train.shape) == 2:
        y = np.argmax(y_train, axis=1)
    accuracy = np.mean(predictions == y)

    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')

    # Backward pass
    loss_activation.backward(loss_activation.output, y_train )
    dense3.backward(loss_activation.dinputs)
    activation2.backward(dense3.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.update_params(dense3)
    optimizer.post_update_params()

(60000, 144)
(60000, 10)
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
epoch: 0, acc: 0.084, loss: 2.303, lr: 0.001
epoch: 100, acc: 0.755, loss: 0.753, lr: 0.0009999010098000298
epoch: 200, acc: 0.889, loss: 0.378, lr: 0.000999801039593121
epoch: 300, acc: 0.919, loss: 0.283, lr: 0.000999701089374277
epoch: 400, acc: 0.937, loss: 0.217, lr: 0.000999601159137504
epoch: 500, acc: 0.951, loss: 0.171, lr: 0.0009995012488768105
epoch: 600, acc: 0.961, loss: 0.139, lr: 0.0009994013585862068
epoch: 700, acc: 0.967, loss: 0.115, lr: 0.0009993014882597065
epoch: 800, acc: 0.973, loss: 0.097, lr: 0.000999201637891325
epoch: 900, acc: 0.977, loss: 0.082, lr: 0.00099910180747508
epoch: 1000, acc: 0.980, loss: 0.070, lr: 0.000999001997004992
epoch: 1100, acc: 0.983, loss: 0.060, lr: 0.0009989022064750839
epoch: 1200, acc: 0.986, loss: 0.052, lr: 0.0009988024358793808
epoch: 1300, acc: 0.988, loss: 0.045, lr: 0.0009987026852119097
epoch: 1400, acc: 0.990, loss: 0.039, lr: 0.0009986029544667011
epoch: 1500, acc: 

32 bit wei and bi qua

In [ ]:
q_dense1 = Quant_Layer_Dense(144, 64)
q_dense1.qw  =dense1.qw.copy()
q_dense1.qb = dense1.qb.copy() # Input size is 12x12 = 144
activation1 = Quant_Activation_ReLU()
q_dense2 = Quant_Layer_Dense(64, 64)
q_dense2.qw = dense2.qw.copy()
q_dense2.qb = dense2.qb.copy()
activation2 = Quant_Activation_ReLU()
q_dense3 =  Quant_Layer_Dense(64, 10)
q_dense3.qw = dense3.qw.copy()
q_dense3.qb = dense3.qb.copy()
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = Quant2_Optimizer_Adam(learning_rate=0.001, decay=1e-6 , k_bit = 32 )

# Training loop


for epoch in range(5001):
    # Perform a forward pass of our training data through this layer
    q_dense1.forward(X_train)

    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(q_dense1.output)

    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    q_dense2.forward(activation1.output)
    activation2.forward(q_dense2.output)
    q_dense3.forward(activation2.output)


    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(q_dense3.output, y_train)

    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y_train.shape) == 2:
        y = np.argmax(y_train, axis=1)
    accuracy = np.mean(predictions == y)

    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')

    # Backward pass
    loss_activation.backward(loss_activation.output, y_train )
    q_dense3.backward(loss_activation.dinputs)
    activation2.backward(q_dense3.dinputs)
    q_dense2.backward(activation2.dinputs)
    activation1.backward(q_dense2.dinputs)
    q_dense1.backward(activation1.dinputs)

    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(q_dense1)
    optimizer.update_params(q_dense2)
    optimizer.update_params(q_dense3)
    optimizer.post_update_params()

epoch: 0, acc: 0.992, loss: 0.034, lr: 0.001
epoch: 100, acc: 0.391, loss: 9.762, lr: 0.0009999010098000298
epoch: 200, acc: 0.396, loss: 9.673, lr: 0.000999801039593121
epoch: 300, acc: 0.401, loss: 9.590, lr: 0.000999701089374277
epoch: 400, acc: 0.406, loss: 9.511, lr: 0.000999601159137504
epoch: 500, acc: 0.411, loss: 9.426, lr: 0.0009995012488768105
epoch: 600, acc: 0.416, loss: 9.343, lr: 0.0009994013585862068
epoch: 700, acc: 0.421, loss: 9.258, lr: 0.0009993014882597065
epoch: 800, acc: 0.426, loss: 9.189, lr: 0.000999201637891325
epoch: 900, acc: 0.428, loss: 9.140, lr: 0.00099910180747508
epoch: 1000, acc: 0.430, loss: 9.114, lr: 0.000999001997004992
epoch: 1100, acc: 0.431, loss: 9.108, lr: 0.0009989022064750839
epoch: 1200, acc: 0.431, loss: 9.108, lr: 0.0009988024358793808
epoch: 1300, acc: 0.431, loss: 9.109, lr: 0.0009987026852119097
epoch: 1400, acc: 0.431, loss: 9.109, lr: 0.0009986029544667011
epoch: 1500, acc: 0.431, loss: 9.109, lr: 0.000998503243637787
epoch: 1600,

KeyboardInterrupt: 

generalized code for all just wei and bia

In [ ]:
for k_bit in [32 , 16 , 8 , 4 , 3  , 2 ]:
  print(k_bit)
  print(['-']*32 )
  q_dense1 = Quant_Layer_Dense(144, 64)
  q_dense1.qw  =dense1.qw.copy()
  q_dense1.qb = dense1.qb.copy() # Input size is 12x12 = 144
  activation1 = Quant_Activation_ReLU()
  q_dense2 = Quant_Layer_Dense(64, 64)
  q_dense2.qw = dense2.qw.copy()
  q_dense2.qb = dense2.qb.copy()
  activation2 = Quant_Activation_ReLU()
  q_dense3 =  Quant_Layer_Dense(64, 10)
  q_dense3.qw = dense3.qw.copy()
  q_dense3.qb = dense3.qb.copy()
  loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
  optimizer = Quant2_Optimizer_Adam(learning_rate=0.001, decay=1e-6 , k_bit = k_bit )

  # Training loop


  for epoch in range(5001):
      # Perform a forward pass of our training data through this layer
      q_dense1.forward(X_train)

      # Perform a forward pass through activation function
      # takes the output of first dense layer here
      activation1.forward(q_dense1.output)

      # Perform a forward pass through second Dense layer
      # takes outputs of activation function of first layer as inputs
      q_dense2.forward(activation1.output)
      activation2.forward(q_dense2.output)
      q_dense3.forward(activation2.output)


      # Perform a forward pass through the activation/loss function
      # takes the output of second dense layer here and returns loss
      loss = loss_activation.forward(q_dense3.output, y_train)

      # Calculate accuracy from output of activation2 and targets
      # calculate values along first axis
      predictions = np.argmax(loss_activation.output, axis=1)
      if len(y_train.shape) == 2:
          y = np.argmax(y_train, axis=1)
      accuracy = np.mean(predictions == y)

      if not epoch % 100:
          print(f'epoch: {epoch}, ' +
                f'acc: {accuracy:.3f}, ' +
                f'loss: {loss:.3f}, ' +
                f'lr: {optimizer.current_learning_rate}')

      # Backward pass
      loss_activation.backward(loss_activation.output, y_train )
      q_dense3.backward(loss_activation.dinputs)
      activation2.backward(q_dense3.dinputs)
      q_dense2.backward(activation2.dinputs)
      activation1.backward(q_dense2.dinputs)
      q_dense1.backward(activation1.dinputs)

      # Update weights and biases
      optimizer.pre_update_params()
      optimizer.update_params(q_dense1)
      optimizer.update_params(q_dense2)
      optimizer.update_params(q_dense3)
      optimizer.post_update_params()


32 bit wei bia and activation qua

In [ ]:
dense1_32 = Quant_Layer_Dense(144, 64)
dense1_32.qw = dense1.qw.copy()
dense1_32.qb = dense1.qb.copy()

activation1 = Quant2_Activation_ReLU()

dense2_32 = Quant_Layer_Dense(64, 64)
dense2_32.qw = dense2.qw.copy()
dense2_32.qb = dense2.qb.copy()

activation2 = Quant2_Activation_ReLU()
dense3_32 =  Quant_Layer_Dense(64 , 10 )

dense3_32.qw = dense3.qw.copy()
dense3_32.qb = dense3.qb.copy()

loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = Quant2_Optimizer_Adam(learning_rate=0.001, decay=1e-6 , k_bit = 32  )

# Training loop


for epoch in range(51):
    # Perform a forward pass of our training data through this layer
    dense1_32.forward(X_train)

    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1_32.output , 32 )

    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2_32.forward(activation1.output)
    activation2.forward(dense2_32.output , 32 )
    dense3_32.forward(activation2.output)


    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense3_32.output, y_train)

    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y_train.shape) == 2:
        y = np.argmax(y_train, axis=1)
    accuracy = np.mean(predictions == y)

    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')

    # Backward pass
    loss_activation.backward(loss_activation.output, y_train )
    dense3_32.backward(loss_activation.dinputs)
    activation2.backward(dense3_32.dinputs)
    dense2_32.backward(activation2.dinputs)
    activation1.backward(dense2_32.dinputs)
    dense1_32.backward(activation1.dinputs)

    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1_32)
    optimizer.update_params(dense2_32)
    optimizer.update_params(dense3_32)
    optimizer.post_update_params()

all aquant

In [ ]:
for k_bit in [ 32 , 16 , 8 , 4 , 3 , 2 ]:
  print(k_bit)
  print(["-"] * 32 )
  dense1_32 = Quant_Layer_Dense(144, 64)
  dense1_32.qw = dense1.qw.copy()
  dense1_32.qb = dense1.qb.copy()

  activation1 = Quant2_Activation_ReLU()

  dense2_32 = Quant_Layer_Dense(64, 64)
  dense2_32.qw = dense2.qw.copy()
  dense2_32.qb = dense2.qb.copy()

  activation2 = Quant2_Activation_ReLU()
  dense3_32 =  Quant_Layer_Dense(64 , 10 )

  dense3_32.qw = dense3.qw.copy()
  dense3_32.qb = dense3.qb.copy()

  loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
  optimizer = Quant2_Optimizer_Adam(learning_rate=0.001, decay=1e-6 , k_bit = k_bit  )

  # Training loop


  for epoch in range(51):
      # Perform a forward pass of our training data through this layer
      dense1_32.forward(X_train)

      # Perform a forward pass through activation function
      # takes the output of first dense layer here
      activation1.forward(dense1_32.output , k_bit  )

      # Perform a forward pass through second Dense layer
      # takes outputs of activation function of first layer as inputs
      dense2_32.forward(activation1.output)
      activation2.forward(dense2_32.output , k_bit  )
      dense3_32.forward(activation2.output)


      # Perform a forward pass through the activation/loss function
      # takes the output of second dense layer here and returns loss
      loss = loss_activation.forward(dense3_32.output, y_train)

      # Calculate accuracy from output of activation2 and targets
      # calculate values along first axis
      predictions = np.argmax(loss_activation.output, axis=1)
      if len(y_train.shape) == 2:
          y = np.argmax(y_train, axis=1)
      accuracy = np.mean(predictions == y)

      if not epoch % 100:
          print(f'epoch: {epoch}, ' +
                f'acc: {accuracy:.3f}, ' +
                f'loss: {loss:.3f}, ' +
                f'lr: {optimizer.current_learning_rate}')

      # Backward pass
      loss_activation.backward(loss_activation.output, y_train )
      dense3_32.backward(loss_activation.dinputs)
      activation2.backward(dense3_32.dinputs)
      dense2_32.backward(activation2.dinputs)
      activation1.backward(dense2_32.dinputs)
      dense1_32.backward(activation1.dinputs)

      # Update weights and biases
      optimizer.pre_update_params()
      optimizer.update_params(dense1_32)
      optimizer.update_params(dense2_32)
      optimizer.update_params(dense3_32)
      optimizer.post_update_params()


GPTQ TECHNIQUE only

In [ ]:
for k_bit in [32 , 16 , 8 , 4 , 2 ]:
  optimizer2 = Quant2_Optimizer_Adam(learning_rate=0.001, decay=1e-6 , k_bit = k_bit )
  print(k_bit)
  print(['-'] *30 )
  for epoch in range(101):

      dense1.forward(X_train)

      # Perform a forward pass through activation function
      # takes the output of first dense layer here
      activation1.forward(dense1.output)

      # Perform a forward pass through second Dense layer
      # takes outputs of activation function of first layer as inputs
      dense2.forward(activation1.output)
      activation2.forward(dense2.output)
      dense3.forward(activation2.output)


      # Perform a forward pass through the activation/loss function
      # takes the output of second dense layer here and returns loss
      loss = loss_activation.forward(dense3.output, y_train)

      # Calculate accuracy from output of activation2 and targets
      # calculate values along first axis
      predictions = np.argmax(loss_activation.output, axis=1)
      if len(y_train.shape) == 2:
          y = np.argmax(y_train, axis=1)
      accuracy = np.mean(predictions == y)

      if not epoch % 100:
          print(f'epoch: {epoch}, ' +
                f'acc: {accuracy:.3f}, ' +
                f'loss: {loss:.3f}, ' +
                f'lr: {optimizer.current_learning_rate}')

      # Backward pass
      loss_activation.backward(loss_activation.output, y_train )
      dense3.backward(loss_activation.dinputs)
      activation2.backward(dense3.dinputs)
      dense2.backward(activation2.dinputs)
      activation1.backward(dense2.dinputs)
      dense1.backward(activation1.dinputs)

      # Update weights and biases
      optimizer2.pre_update_params()
      optimizer2.update_params(dense1)
      optimizer2.update_params(dense2)
      optimizer2.update_params(dense3)
      optimizer2.post_update_params()

In [ ]:
import larq as lq
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Fetch the MNIST dataset
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist['data'], mnist['target']
X = X.to_numpy()
# Reshape to 28x28 and then remove edge rows/cols to get 24x24
X = X.reshape(-1, 28, 28)
X = X[:, 2:-2, 2:-2]  # Remove 2 rows and cols from each side -> 24x24

# Take alternate pixels to downsample to 12x12
X = X[:, ::2, ::2]  # Now 12x12
X = X.reshape(-1, 12 * 12)  # Flatten the images

# Normalize the data
X = X / 255.0
y = np.array(y).astype(int)

# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False, categories='auto') # sparse has been replaced by sparse_output
y = encoder.fit_transform(y.reshape(-1, 1))

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Network architecture
input_size = 12 * 12  # 12x12 images
hidden_layer1_size = 64  # Reduced number of neurons
hidden_layer2_size = 64  # Reduced number of neurons
output_size = 10  # Digits 0-9

# Weight initialization
def initialize_weights(input_size, hidden_layer1_size, hidden_layer2_size, output_size):
    W1 = np.random.randn(input_size, hidden_layer1_size) * np.sqrt(2. / input_size)
    b1 = np.zeros((1, hidden_layer1_size))
    W2 = np.random.randn(hidden_layer1_size, hidden_layer2_size) * np.sqrt(2. / hidden_layer1_size)
    b2 = np.zeros((1, hidden_layer2_size))
    W3 = np.random.randn(hidden_layer2_size, output_size) * np.sqrt(2. / hidden_layer2_size)
    b3 = np.zeros((1, output_size))
    return W1, b1, W2, b2, W3, b3

# Activation functions
def relu(Z):
    return np.maximum(0, Z)

def relu_derivative(Z):
    return Z > 0

def softmax(Z):
    exp_values = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return exp_values / np.sum(exp_values, axis=1, keepdims=True)

# Loss function
def cross_entropy_loss(y_true, y_pred):
    m = y_true.shape[0]
    loss = -np.sum(y_true * np.log(y_pred + 1e-9)) / m
    return loss

# Forward pass
def forward_pass(X, W1, b1, W2, b2, W3, b3):
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = relu(Z2)
    Z3 = np.dot(A2, W3) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2, Z3, A3

# Quantization functions
def quantize_weights(matrix, k_bit):
    quantizer = lq.quantizers.DoReFaQuantizer(k_bit=k_bit, mode="weights")
    return quantizer(matrix)

def quantize_bias(matrix, k_bit):
    quantizer = lq.quantizers.DoReFaQuantizer(k_bit=k_bit, mode="activations")
    return quantizer(matrix)

# Backward pass
def backward_pass(X, y, Z1, A1, Z2, A2, Z3, A3, W1, W2, W3):
    m = y.shape[0]

    # Output layer error
    dZ3 = A3 - y
    dW3 = np.dot(A2.T, dZ3) / m
    db3 = np.sum(dZ3, axis=0, keepdims=True) / m

    # Hidden layer 2 error
    dA2 = np.dot(dZ3, W3.T)
    dZ2 = dA2 * relu_derivative(Z2)
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    # Hidden layer 1 error
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    return dW1, db1, dW2, db2, dW3, db3

# Adam optimizer
class AdamOptimizer:
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = {}
        self.v = {}
        self.t = 0

    def update_params(self, params, grads):
        if not self.m:
            self.m = {k: np.zeros_like(v) for k, v in params.items()}
            self.v = {k: np.zeros_like(v) for k, v in params.items()}

        self.t += 1
        updated_params = {}

        for k in params.keys():
            self.m[k] = self.beta1 * self.m[k] + (1 - self.beta1) * grads[k]
            self.v[k] = self.beta2 * self.v[k] + (1 - self.beta2) * (grads[k] ** 2)

            m_hat = self.m[k] / (1 - self.beta1 ** self.t)
            v_hat = self.v[k] / (1 - self.beta2 ** self.t)

            updated_params[k] = params[k] - self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)

        return updated_params

# Training the model
# def train(X_train, y_train, X_test, y_test, learning_rate=0.001, epochs=500 , k_bit =64 ):
#     W1, b1, W2, b2, W3, b3 = initialize_weights(input_size, hidden_layer1_size, hidden_layer2_size, output_size)
#     qw1, qw2, qw3 = W1.copy(), W2.copy(), W3.copy()
#     qb1, qb2, qb3 = b1.copy(), b2.copy(), b3.copy()
#     optimizer = AdamOptimizer(learning_rate)
#     print(k_bit)
#     print(["-"]*32)
#     for epoch in range(epochs):
#         # Forward pass
#         Z1, A1, Z2, A2, Z3, A3 = forward_pass(X_train, W1, b1, W2, b2, W3, b3)

#         # Calculate loss
#         loss = cross_entropy_loss(y_train, A3)

#         # Backward pass
#         # dW1, db1, dW2, db2, dW3, db3 = backward_pass(X_train, y_train, Z1, A1, Z2, A2, Z3, A3, qw1, qw2, qw3)
#         dW1, db1, dW2, db2, dW3, db3 = backward_pass(X_train, y_train, Z1, A1, Z2, A2, Z3, A3, W1, W2, W3)

#         # Update weights and biases using Adam optimizer
#         params = {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2, 'W3': W3, 'b3': b3}
#         grads = {'W1': dW1, 'b1': db1, 'W2': dW2, 'b2': db2, 'W3': dW3, 'b3': db3}
#         updated_params = optimizer.update_params(params, grads)

#         qw1, qb1, qw2, qb2, qw3, qb3 = updated_params['W1'], updated_params['b1'], updated_params['W2'], updated_params['b2'], updated_params['W3'], updated_params['b3']

#         # Apply quantization for different bit sizes

#         W1, W2, W3 = quantize_weights(qw1.copy(), k_bit), quantize_weights(qw2.copy(), k_bit), quantize_weights(qw3.copy(), k_bit)
#         b1, b2, b3 = quantize_bias(qb1.copy(), k_bit), quantize_bias(qb2.copy(), k_bit), quantize_bias(qb3.copy(), k_bit)

#         if (epoch + 1) % 100 == 0:
#                 _, _, _, _, _, A3_test = forward_pass(X_test, W1, b1, W2, b2, W3, b3)
#                 predictions = np.argmax(A3_test, axis=1)
#                 labels = np.argmax(y_test, axis=1)
#                 accuracy = np.mean(predictions == labels) * 100
# Training the model
def train(X_train, y_train, X_test, y_test, learning_rate=0.001, epochs=500 , k_bit =64 ):
    W1, b1, W2, b2, W3, b3 = initialize_weights(input_size, hidden_layer1_size, hidden_layer2_size, output_size)
    qw1, qw2, qw3 = W1.copy(), W2.copy(), W3.copy()
    qb1, qb2, qb3 = b1.copy(), b2.copy(), b3.copy()
    optimizer = AdamOptimizer(learning_rate)
    print(k_bit)
    print(["-"]*32)
    for epoch in range(epochs):
        # Forward pass
        Z1, A1, Z2, A2, Z3, A3 = forward_pass(X_train, W1, b1, W2, b2, W3, b3)

        # Calculate loss
        loss = cross_entropy_loss(y_train, A3)

        # Backward pass
        # dW1, db1, dW2, db2, dW3, db3 = backward_pass(X_train, y_train, Z1, A1, Z2, A2, Z3, A3, qw1, qw2, qw3)
        dW1, db1, dW2, db2, dW3, db3 = backward_pass(X_train, y_train, Z1, A1, Z2, A2, Z3, A3, W1, W2, W3)

        # Update weights and biases using Adam optimizer
        params = {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2, 'W3': W3, 'b3': b3}
        grads = {'W1': dW1, 'b1': db1, 'W2': dW2, 'b2': db2, 'W3': dW3, 'b3': db3}
        updated_params = optimizer.update_params(params, grads)

        qw1, qb1, qw2, qb2, qw3, qb3 = updated_params['W1'], updated_params['b1'], updated_params['W2'], updated_params['b2'], updated_params['W3'], updated_params['b3']

        # Apply quantization for different bit sizes

        W1, W2, W3 = quantize_weights(qw1.copy(), k_bit), quantize_weights(qw2.copy(), k_bit), quantize_weights(qw3.copy(), k_bit) # Removed .numpy() as they are already NumPy arrays
        b1, b2, b3 = quantize_bias(qb1.copy(), k_bit), quantize_bias(qb2.copy(), k_bit), quantize_bias(qb3.copy(), k_bit) # Removed .numpy() as they are already NumPy arrays
        if (epoch + 1) % 100 == 0:
                _, _, _, _, _, A3_test = forward_pass(X_test, W1, b1, W2, b2, W3, b3)
                predictions = np.argmax(A3_test, axis=1)
                labels = np.argmax(y_test, axis=1)
                accuracy = np.mean(predictions == labels) * 100

                print(f"Epoch {epoch + 1}: Accuracy = {accuracy:.2f}%")

for k_bit in [32 , 16 , 8 , 4 , 2 ]:
  train(X_train, y_train, X_test, y_test , k_bit )

64
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']


AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'copy'

my code again

In [ ]:
import larq as lq
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Fetch the MNIST dataset
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist['data'], mnist['target']

# Normalize the data
X = X / 255.0
y = np.array(y).astype(int)

# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False, categories='auto')
y = encoder.fit_transform(y.reshape(-1, 1))

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Network architecture
input_size = 784  # 28x28 images
hidden_layer1_size = 512  # Increased number of neurons
hidden_layer2_size = 256  # Increased number of neurons
output_size = 10  # Digits 0-9

# Weight initialization

def initialize_weights(input_size, hidden_layer1_size, hidden_layer2_size, output_size):
    W1 = np.random.randn(input_size, hidden_layer1_size) * np.sqrt(2. / input_size)
    b1 = np.zeros((1, hidden_layer1_size))
    W2 = np.random.randn(hidden_layer1_size, hidden_layer2_size) * np.sqrt(2. / hidden_layer1_size)
    b2 = np.zeros((1, hidden_layer2_size))
    W3 = np.random.randn(hidden_layer2_size, output_size) * np.sqrt(2. / hidden_layer2_size)
    b3 = np.zeros((1, output_size))
    return W1, b1, W2, b2, W3, b3

# Activation functions
def relu(Z):
    return np.maximum(0, Z)

def relu_derivative(Z):
    return Z > 0

def softmax(Z):
    exp_values = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return exp_values / np.sum(exp_values, axis=1, keepdims=True)

# Loss function
def cross_entropy_loss(y_true, y_pred):
    m = y_true.shape[0]
    loss = -np.sum(y_true * np.log(y_pred + 1e-9)) / m
    return loss



# Forward pass
def forward_pass(X, W1, b1, W2, b2, W3, b3):
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = relu(Z2)
    Z3 = np.dot(A2, W3) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2, Z3, A3


def quantize_weights( matrix  , k_bit ):
  quantizer = lq.quantizers.DoReFaQuantizer(k_bit= k_bit, mode="weights")
  return quantizer(matrix)

def quantize_bias( matrix , k_bit):
  quantizer = lq.quantizers.DoReFaQuantizer(k_bit=k_bit , mode="activations")
  return quantizer(matrix )

# Backward pass
def backward_pass(X, y, Z1, A1, Z2, A2, Z3, A3, W1, W2, W3):
    m = y.shape[0]

    # Output layer error
    dZ3 = A3 - y
    dW3 = np.dot(A2.T, dZ3) / m
    db3 = np.sum(dZ3, axis=0, keepdims=True) / m

    # Hidden layer 2 error
    dA2 = np.dot(dZ3, W3.T)
    dZ2 = dA2 * relu_derivative(Z2)
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    # Hidden layer 1 error
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    return dW1, db1, dW2, db2, dW3, db3

# Adam optimizer
class AdamOptimizer:
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = {}
        self.v = {}
        self.t = 0

    def update_params(self, params, grads):
        if not self.m:
            self.m = {k: np.zeros_like(v) for k, v in params.items()}
            self.v = {k: np.zeros_like(v) for k, v in params.items()}

        self.t += 1
        updated_params = {}

        for k in params.keys():
            self.m[k] = self.beta1 * self.m[k] + (1 - self.beta1) * grads[k]
            self.v[k] = self.beta2 * self.v[k] + (1 - self.beta2) * (grads[k] ** 2)

            m_hat = self.m[k] / (1 - self.beta1 ** self.t)
            v_hat = self.v[k] / (1 - self.beta2 ** self.t)

            updated_params[k] = params[k] - self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)

        return updated_params

# Training the model
def train(X_train, y_train, X_test, y_test, learning_rate=0.001, epochs=51 , k_bit = 64 ):
    W1, b1, W2, b2, W3, b3 = initialize_weights(input_size, hidden_layer1_size, hidden_layer2_size, output_size)
    qw1 = W1.copy() ; qw2 = W2.copy() ; qw3 = W3.copy() ; qb1 = b1.copy() ; qb2 = b2.copy() ; qb3 = b3.copy() ;
    # taken the
    # qw1 = W1.copy() ; qb1 =  b1.copy() ; qw2 = W2.copy() ; qb2 =  b2.copy() ; qw3 = W3.copy() ; qb3 =  b3.copy()
    optimizer = AdamOptimizer(learning_rate)
    print(k_bit)
    print(["-"]*32)
    for epoch in range(epochs):
        # Forward pass
        Z1, A1, Z2, A2, Z3, A3 = forward_pass(X_train, W1, b1, W2, b2, W3, b3)

        # Calculate loss
        loss = cross_entropy_loss(y_train, A3)

        # Backward pass
        dW1, db1, dW2, db2, dW3, db3 = backward_pass(X_train, y_train, Z1, A1, Z2, A2, Z3, A3, qw1, qw2, qw3)

        # Update weights and biases using Adam optimizer
        params = {'W1': qw1, 'b1': qb1, 'W2': qw2, 'b2': qb2, 'W3': qw3, 'b3': qb3}
        grads = {'W1': dW1, 'b1': db1, 'W2': dW2, 'b2': db2, 'W3': dW3, 'b3': db3}
        updated_params = optimizer.update_params(params, grads)

        qw1, qb1, qw2, qb2, qw3, qb3 = updated_params['W1'], updated_params['b1'], updated_params['W2'], updated_params['b2'], updated_params['W3'], updated_params['b3']
        W1 , W2  , W3  = quantize_weights(k_bit =k_bit, matrix = qw1.copy() )  , quantize_weights(k_bit = k_bit , matrix = qw2.copy() )  , quantize_weights(k_bit = k_bit , matrix = qw3.copy() )
        b1  , b2  , b3 = quantize_bias(k_bit=k_bit , matrix  = qb1.copy()) , quantize_bias(k_bit =k_bit, matrix = qb2.copy()) , quantize_bias(k_bit = k_bit, matrix = qb3.copy())

        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}')

    _, _, _, _, _, A3_test = forward_pass(X_test, W1, b1, W2, b2, W3, b3)
    # print()
    predictions = np.argmax(A3_test, axis=1)
    labels = np.argmax(y_test, axis=1)
    accuracy = np.mean(predictions == labels) * 100
    print(f'Test Accuracy: {accuracy:.2f}%')






# Train the network
for k_bit in [32  , 16 , 8 , 4 , 2]:
  train(X_train = X_train  , y_train = y_train , X_test = X_test , y_test = y_test , k_bit = k_bit )

32
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
Epoch 10/51, Loss: 1.6340
Epoch 20/51, Loss: 0.8052
Epoch 30/51, Loss: 0.4625
Epoch 40/51, Loss: 0.3232
Epoch 50/51, Loss: 0.2408
Test Accuracy: 93.41%
16
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
Epoch 10/51, Loss: 1.7244
Epoch 20/51, Loss: 0.8828
Epoch 30/51, Loss: 0.5537
Epoch 40/51, Loss: 0.3858
Epoch 50/51, Loss: 0.2889
Test Accuracy: 92.84%
8
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
Epoch 10/51, Loss: 1.5155
Epoch 20/51, Loss: 0.8209
Epoch 30/51, Loss: 0.5427
Epoch 40/51, Loss: 0.3724
Epoch 50/51, Loss: 0.2763
Test Accuracy: 93.56%
4
['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', 